In [2]:
import pandas
import sys
from utils import jaccard_similarity, load_data

In [4]:
dataset = load_data("./active1000")

In [5]:
# Remove unused columns.
dataset.drop(columns=["eventId", "category", "activeTime", "title", "publishtime", "time", "documentId"], inplace=True)

# Remove events related to the home page.
dataset.drop(index=dataset[dataset.url == "http://adressa.no"].index, inplace=True)

print(dataset)

                                                       url  \
2        http://adressa.no/nyheter/trondheim/2016/12/31...   
4        http://adressa.no/pluss/kultur/2016/12/31/bare...   
15       http://adressa.no/nyheter/2016/12/31/se-lesern...   
16       http://adressa.no/nyheter/trondheim/2016/12/31...   
17       http://adressa.no/nyheter/2016/12/31/se-lesern...   
...                                                    ...   
2207591  http://adressa.no/nyheter/moreromsdal/2017/03/...   
2207598  http://adressa.no/nyheter/trondheim/2017/03/31...   
2207599  http://adressa.no/nyheter/trondheim/2017/03/31...   
2207603  http://adressa.no/pluss/magasin/2017/03/31/tre...   
2207607  http://adressa.no/incoming/2017/03/31/transpar...   

                                              userId  
2                  cx:iimz2wwcwxu7d721:2r8odp9zhg5yp  
4                  cx:iimz2wwcwxu7d721:2r8odp9zhg5yp  
15                 cx:il0sdznsjgg9uxgy:3bi2ksost85yi  
16                  cx:ihnzu06beuaz

In [15]:
# Create sets of articles per user
userSets = dataset.groupby(by="userId")["url"].apply(set)

userId
cx:10k2wzm1b3jsk2y5ym1utnjo97:2kefzqaxe9jx7    {http://adressa.no/100sport/vintersport/langre...
cx:11asuyo07mbi13b849lp87hlje:1i9z1hu1xx694    {http://adressa.no/nyheter/innenriks/2017/03/1...
cx:13077170178592105431908:2so1nc52xur4d       {http://adressa.no/pluss/kultur/2017/01/23/cre...
cx:13082126012361420762846:3nhzg71lk2zqc       {http://adressa.no/nyheter/nordtrondelag/2017/...
cx:13082926635761580649288:30q2cgmwa3mzi       {http://adressa.no/pluss/2016/03/07/krever-vei...
                                                                     ...                        
cx:ymgd4qz84t6z31b22zecmrpz3:2bclyyq01sfwd     {http://adressa.no/reise/ny-prestisjekaring-de...
cx:ywxiqgtmxyfc24eqhyzpz7jer:28kehwib7kyi8     {http://adressa.no/pluss/nyheter/2017/02/16/ha...
cx:z5nmgvym79d91yxajr8rncjp3:dd23bgkvv19x      {http://adressa.no/folk/2017/03/22/minneord-pe...
cx:zdxhidle29xquzztafvsik92:2w17zz40qj619      {http://adressa.no/nyheter/trondheim/2017/03/2...
cx:ztquyfd3pug92dd4jpqz

In [20]:
sim_matrix = {}

for user, user_set in userSets.iteritems():
    for other_user, other_user_set in userSets.iteritems():
        jacc = jaccard_similarity(user_set, other_user_set)
        print(jacc)
        if user in sim_matrix:
            sim_matrix[user][other_user] = jacc
        else:
            sim_matrix[user] = {}
            sim_matrix[user][other_user] = jacc

1.0


SystemExit: 0